<a href="https://colab.research.google.com/github/tlyddiard/keras_models/blob/main/EMSPy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install git+https://github.com/mattsq/emspy.git#egg=emspy

  Cloning https://github.com/mattsq/emspy.git to /tmp/pip-install-5xjo1k9x/emspy
  Running command git clone -q https://github.com/mattsq/emspy.git /tmp/pip-install-5xjo1k9x/emspy
  Created wheel for emspy: filename=emspy-0.4.1-cp36-none-any.whl size=43830 sha256=90454554396d24d787d1338b6e3d9bd098cdec18df50680324c16cfc0bf5dc4d
  Stored in directory: /tmp/pip-ephem-wheel-cache-a6ug7v65/wheels/ce/1b/44/7ee0010227577f206312cede8a2b63216767db5c68394896cd
Successfully built emspy


In [ ]:
user_name = 'thomas.lyddiard'
pwd = 'october2020#'

In [ ]:
import emspy
from emspy.query import FltQuery

c = emspy.Connection(user= user_name,pwd=pwd, server_url="https://qfa-api.au.efoqa.com/api")

query = emspy.query.FltQuery(conn=c, ems_name="QANTAS-EMS", data_file="metadata.db") 

query.update_dbtree("APM Events", "Qantas Airways Development", "P14")
query.set_database("P14")
query.generate_preset_fieldtree()

query.update_fieldtree("Profile 14: event information")
query.update_fieldtree("Profile 14: measurements for all events","timepoints")
query.update_fieldtree("Profile 14: measurements for all events","measurements", "data quality")
query.update_fieldtree("Profile 14: measurements for all events","measurements","time","durations", "actual")
query.update_fieldtree("Profile 14: measurements for all events","measurements","time", "gmt")
query.update_fieldtree("Profile 14: measurements for all events","measurements","navigation", "latitude")
query.update_fieldtree("Profile 14: measurements for all events","measurements","navigation", "Terrain", "MSL (above mean sea level)")
query.update_fieldtree("Profile 14: measurements for all events","measurements","flight control", "commands")
query.update_fieldtree("Profile 14: measurements for all events","measurements","flight control", "surfaces")

query.select("flight record",
            "flight date (exact)",
            "tail number",
            "airframe",
            "fleet",
            "takeoff airport code",
            "landing airport code",
            "p14: event record",
            "p14: event type",
            "p14: severity",
            "p14: false positive",
            "p14: start of event - 10 seconds",
            "p14: end of event + 10 seconds",
            "p14: data dropouts",
            "p14: error b/w GMT and Recorded Time",
            "p14: estimated nav error",
            "p14: standard deviation of sample",
            "p14: flap handle position",
            "p14: speed brake handle position",
            "p14: flap position at start",
            "p14: Duration of Event (Seconds)",
            "p14: Terrain Elevation at start of event (ft)")
                                        
                                        
query.filter("'flight date (exact)' >= '2019-12-01'")
query.filter("'flight date (exact)' < '2020-12-31'")
query.filter("'fleet' == 'QFA - 737-800 (1024 wps)'") 
query.filter("'P14: Severity' in ['Level 2','Level 3']")                                        

query.get_top(n=5000)
raw = query.run()

clean = raw.copy()

elements = [
  'Bad Parameter: Bad Sensor on Airplane', 
  'Bad Parameter: Cause Unknown', 
  'Bad Parameter: Flight Recorder Error',
  'Bad Parameter: Short-term Data Dropout',
  'Bad Parameter: Synchronization Error',
  'Not a False Positive']

clean = clean[clean['P14: False Positive'].isin(elements)]

target_dict = {
  'Bad Parameter: Bad Sensor on Airplane': 1,
  'Bad Parameter: Cause Unknown': 1,
  'Bad Parameter: Flight Recorder Error': 1,
  'Bad Parameter: Short-term Data Dropout': 1,
  'Bad Parameter: Synchronization Error': 1,
  'Not a False Positive': 0
}

clean['target'] = clean['P14: False Positive'].map(target_dict)


On <root>(root)...
=== Starting to update subtree from 'FDW (database_group)' ===
On FDW(database_group)...
-- Added 3 databases
On Dimensions(database_group)...
-- Added 20 databases
On APM Events(database_group)...
On Qantas Airways Development Profiles(database_group)...
=== Starting to update subtree from 'P14: QANTAS Safety Events (database_group)' ===
On P14: QANTAS Safety Events(database_group)...
-- Added 169 databases
On P14: QANTAS Safety Events - All Events(database)...
Using database 'P14: QANTAS Safety Events - All Events'.
On P14: QANTAS Safety Events - All Events(database)...
On Flight Information(field_group)...
-- Added 9 fields
On Date Times(field_group)...
-- Added 4 fields
On Aircraft Information(field_group)...
-- Added 22 fields
On Airframe Information(field_group)...
-- Added 4 fields
On Engine Information(field_group)...
On Best Available Engine Information(field_group)...
-- Added 30 fields
On GE Core Engine Data(field_group)...
On Left Outbd Engine(field_group

In [ ]:
clean.head()

,Flight Record,Flight Date (Exact),Tail Number,Airframe,Fleet,Takeoff Airport Code,Landing Airport Code,P14: Event Record,P14: Event Type,P14: Severity,P14: False Positive,P14: Start of Event - 10 seconds Seconds (From Start of File),P14: End of Event + 10 seconds Seconds (From Start of File),P14: Data Dropouts (seconds),P14: Error b/w GMT and Recorded Time around event (%),P14: Estimated Nav Error at Start of Event (nm),P14: Standard Deviation of Sample Rate of GMT (Event Interval +/- 30 Seconds (Hz),P14: Flap Handle Position at Start of Event,P14: Speed Brake Handle Position at Start of Event (deg),P14: Flap Position at Start of Event (deg+=TED),P14: Duration of Event (Seconds),P14: Terrain Elevation at Start of Event (ft),target
0,2869387,2020-02-25 06:13:53+00:00,VH-VXD,737-800,QFA - 737-800 (1024 wps),YPPH,YPAD,40509975,Airborne Speed High,Level 2,Not a False Positive,609.00,629.00,0.0,-0.000095,0.024290,0.0,5.0,0.641989,5.203770,0.0,56.428040,0
1,2869366,2020-02-25 07:03:25+00:00,VH-VYF,737-800,QFA - 737-800 (1024 wps),YMML,YSSY,40509984,High Vertical Acceleration at Landing,Level 2,Not a False Positive,5611.25,5637.25,2.0,-0.000436,0.038936,0.0,30.0,4.200743,30.009817,6.0,12.124564,0
2,2869228,2020-02-25 03:32:44+00:00,VH-XZN,737-800,QFA - 737-800 (1024 wps),YNWN,YPPH,40510130,Airborne Speed High,Level 2,Not a False Positive,688.00,708.00,0.0,-0.000095,0.009928,0.0,10.0,-1.286268,10.302753,0.0,1713.618530,0
4,2869620,2020-02-25 17:05:39+00:00,VH-VZK,737-800,QFA - 737-800 (1024 wps),NZAA,YMML,40510312,Pitch Attitude High During Rotation (Gears Ext...,Level 2,Not a False Positive,662.00,682.00,0.0,-0.001526,0.032998,0.0,1.0,-1.194254,1.030449,0.0,16.776936,0
6,2868605,2020-02-24 17:17:54+00:00,VH-XZG,737-800,QFA - 737-800 (1024 wps),NZCH,YSSY,40511060,TCAS Resolution Advisory,Level 2,Not a False Positive,10624.00,10663.00,2.0,-0.001691,0.008319,0.0,5.0,-1.654518,5.180016,19.0,-16.391178,0


In [ ]:
filename = 'Flight-Level-False-Pos-Data_new.csv'

clean.to_csv(filename, index = True, header=True)

In [ ]:
from google.colab import files
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>